# **Analyse de sentiments sur les critiques spectateurs sur Allociné**
**Projet Python pour la Data science - 2A ENSAE**

Zakaria BOULLIAIRE, Massyle DENDENE, Brian RAMESH

# Introduction

L'idée de ce projet est de prédire ("mettre la variable à prédire"), à partir d'une analyse de sentiment faite sur les critiques données pas les spectateurs (et non la presse), sur le site Allociné. 
Nous allons donc consituer une base de données de film, en scrappant le site Allociné et en utilisant l'API de The Movie Database (TMDB) pour compléter les données manquantes.

### Imports nécessaires

In [3]:
%%time
# BeutifulSoup pour le scrapping
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import requests

# Classique python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint

from warnings import warn
from time import sleep

import seaborn as sns

from datetime import datetime
from dateutil import parser 

CPU times: user 1.76 s, sys: 4.21 s, total: 5.98 s
Wall time: 1.02 s


# Collecte des données

### Scrapping sur Allociné

Tout d'abord, nous allons restreindre notre base de données au film américain des années 2010 à 2021. Cela constiuerait une base de données de 11 822 films.

Les données qui peuvent nous être utiles sont les suivantes : Titre original du film, identifiant du film sur Allociné (qui nous sera utile pour récupérer les critiques plus tard), la note des spectateurs et celle de la presse, le nombre de critiques presse, le nombre de critiques spectateurs et le nombre de votes pour la notes (dans une seule et même variable, qu'on spérera lors du nettoyage), la date de sortie du film, le budget et le Box Office US. 
Nous avions également commencer à scrapper le N°Visa du film, qui est unique pour chaque film, en vue d'utiliser cette donnée pour la collecte sur TMDB, mais il y avait énormément de film pour lesquels cette donnée manquait. Nous avons décider de ne pas l'utiliser.

L'idée de la fonction ci dessous est de scrapper les données dont on a besoin à partir de l'url et du nombre de page.

In [4]:
%%time
def scrapping_film_allocine(base_url, nb_page):

    # Liste ou on stock nos données
    data = []
    
    # Variable de comptage pour voir l'évolution du scrapping, et détecter les eventuelles erreurs
    i=0

    # Boucle sur les pages
    for page in range(1, nb_page+1): 
        url_page_ac = f"{base_url}{page}"

        response_page_ac = requests.get(url_page_ac)

        if response_page_ac.status_code == 200:
            bs_page_ac = bs(response_page_ac.text, "html.parser")
            films_page_ac = bs_page_ac.findAll("li", attrs={'class': "mdl"})

            for film_allocine in films_page_ac:
                i+=1
                try:
                    
                    # Id du film sur allo cine
                    meta_title_link = film_allocine.find('a', class_='meta-title-link')

                    if meta_title_link:
                        href1 = meta_title_link.get('href')
                        film_id = href1.split('=')[-1].split('.')[0]
                    else:
                        film_id = None


                    # Scrapping de la page fiche info du film qu'on obtient grace à l'id trouvé ci dessus
                    url_fiche_film = f'https://www.allocine.fr/film/fichefilm_gen_cfilm={film_id}.html'
                    response_fiche_film = requests.get(url_fiche_film)
                    bs_fiche_film = bs(response_fiche_film.text, "html.parser")

                    # Titre
                    titre_allocine = meta_title_link.text

                    span_titre_original = bs_fiche_film.find('span', class_='light', string='Titre original ')
                    titre_original = span_titre_original.find_next_sibling(string=True).strip() if span_titre_original else titre_allocine


                    # Notes spectateurs/presse, nombre critiques presse/spectateurs
                    bloc_notes = bs_fiche_film.findAll('span', class_='stareval-note')
                    list_notes = [notes.get_text(strip=True) for notes in bloc_notes]

                    if len(list_notes)==0:
                        note_presse = None
                        note_spectateur = None
                        
                    else:
                        index_delimiteur = list_notes.index('--')
                        new_liste_notes = list_notes[:index_delimiteur]

                        if len(new_liste_notes)==2:
                            note_presse = new_liste_notes[0]
                            note_spectateur = new_liste_notes[1]
                        
                        elif len(new_liste_notes) > 0 and len(new_liste_notes) <= 1:
                            note_spectateur = new_liste_notes[0]
                            note_presse = None

                    bloc_critiques = bs_fiche_film.find_all('span', class_='stareval-review')

                    if len(bloc_critiques)==2:
                        critiques_element_presse = bloc_critiques[0].text
                        critiques_element_spec = bloc_critiques[1].text
                    elif len(bloc_critiques) > 0 and len(bloc_critiques) <= 1:
                        critiques_element_spec = bloc_critiques[0].text
                        critiques_element_presse = None
                    else:
                        critiques_element_presse = None
                        critiques_element_spec = None


                    #Date, durée, budget
                    date_film_element = film_allocine.find('span', class_='date')
                    date_film = date_film_element.text if date_film_element else None

                    duree_film_element = bs_fiche_film.find('span', class_='spacer')
                    duree_film = duree_film_element.next_sibling.strip() if duree_film_element else None

                    budget_element = bs_fiche_film.find('span', class_='what light', string='Budget')
                    budget_film = budget_element.find_next('span').string if budget_element else None

                    #visa_element = bs_fiche_film.find('span', class_='what light', string='N° de Visa')
                    #visa_film = visa_element.find_next('span').string if visa_element else None

                    # Box office
                    url_box_office = f'https://www.allocine.fr/film/fichefilm-{film_id}/box-office/'
                    response_box_office = requests.get(url_box_office)
                    bs_box_office = bs(response_box_office.text, "html.parser")

                    cumul = bs_box_office.findAll('td', {'data-heading': 'Cumul'})
                    list_cumul = [cum.get_text(strip=True) for cum in cumul]

                    box_office_film = list_cumul[-1] if list_cumul else 'None'

                    data.append([titre_original, note_presse, note_spectateur, critiques_element_presse, critiques_element_spec, film_id, box_office_film, budget_film,
                                date_film, duree_film])
                    
                    df_data = pd.DataFrame(data, columns=["Titre original", "Note press", "Notes spectateur", "Critiques presse", "Critiques spectateurs", 'id allocine',
                                          'Box office', 'Budget', 'date', 'duree'])


                    print(i)
                except Exception as e:
                    print(f"Une erreur s'est produite pour le film {i} : {e}")

    return df_data

CPU times: user 4 µs, sys: 5 µs, total: 9 µs
Wall time: 13.8 µs


In [ ]:
%%time

# Urls qu'on veut scrapper
base_url_2010_2020, nb_page_2010_2020 = 'https://www.allocine.fr/films/pays-5002/decennie-2010/?page=', 646
base_url_2020, nb_page_2020 = 'https://www.allocine.fr/films/pays-5002/decennie-2020/annee-2020/?page', 66
base_url_2021, nb_page_2021 = 'https://www.allocine.fr/films/pays-5002/decennie-2020/annee-2021/?page' 77


# Application de la foncrion scrapping_film_allocine
df_data_2010_2020 = scrapping_film_allocine(base_url_2010_2020, nb_page_2010_2020)
df_data_2020 = scrapping_film_allocine(base_url_2020, nb_page_2020)
df_data_2021 = scrapping_film_allocine(base_url_2021, nb_page_2021)

# Créer un DataFrame avec les données collectées
dfs=[df_data_2010_2020, df_data_2020, df_data_2021]

df_film_ac = pd.concat(dfs, ignore_index=True)

In [ ]:
df_film_ac.to_csv('df_film_ac.csv', index=False)

### Nettoyage de la base obtenue sur Allociné

In [19]:
df_film_ac= pd.read_csv("df_film_ac.csv")

In [20]:
df_film_ac 

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree
0,Shutter Island,"3,8","4,4",31 critiques,80471 notes dont 4605 critiques,132039,127 770 000,80 000 000 $,24 février 2010,2h 17min
1,Inception,"4,1","4,5",24 critiques,110095 notes dont 7212 critiques,143692,290 948 208,160 000 000 $,21 juillet 2010,2h 28min
2,Harry Potter and the Deathly Hallows - Part 1,"3,2","4,0",20 critiques,52676 notes dont 2887 critiques,126693,291 377 000,150 000 000 $,24 novembre 2010,2h 26min
3,Prince of Persia: The Sands of Time,"2,6","3,1",22 critiques,26730 notes dont 2133 critiques,126678,89 981 000,200 000 000 $,26 mai 2010,2h 06min
4,The Book of Eli,"2,4","3,3",20 critiques,10503 notes dont 1144 critiques,128955,92 524 000,80 000 000 $,20 janvier 2010,1h 49min
...,...,...,...,...,...,...,...,...,...,...
11816,Amarillo,NaN,NaN,NaN,NaN,294872,NaN,-,NaN,NaN
11817,Chastise,NaN,NaN,NaN,NaN,295695,NaN,-,NaN,NaN
11818,Tapawingo,NaN,NaN,NaN,NaN,296176,NaN,-,NaN,NaN
11819,Mr. Birthday,NaN,NaN,NaN,NaN,298844,NaN,-,NaN,NaN


Comme on peut voir, la base a besoin d'être nettoyé. Nous allons extraires le nombre de critiques de la presse, le nombre de notes sepcteurs et le nombre de critiques des spectateurs. Nous allons convertir également le format de la date, et la durée du film en minute.

Puis, nous allons supprimer tous les films qui n'ont pas de note spectateurs et/ou qui ont moins de 5 critiques

In [21]:
print(df_film_ac.dtypes)

Titre original           object
Note press               object
Notes spectateur         object
Critiques presse         object
Critiques spectateurs    object
id allocine               int64
Box office               object
Budget                   object
date                     object
duree                    object
dtype: object


In [22]:
# Modifications des types 

df_film_ac['Box office'] = df_film_ac['Box office'].str.replace(' ', '').astype(float)
df_film_ac['Note press'] = df_film_ac['Note press'].str.replace(',', '.').astype(float)
df_film_ac['Notes spectateur'] = df_film_ac['Notes spectateur'].replace("--", np.nan)
df_film_ac['Notes spectateur'] = df_film_ac['Notes spectateur'].str.replace(',', '.').astype(float)

In [23]:
print(df_film_ac.dtypes)

Titre original            object
Note press               float64
Notes spectateur         float64
Critiques presse          object
Critiques spectateurs     object
id allocine                int64
Box office               float64
Budget                    object
date                      object
duree                     object
dtype: object


In [24]:
# Fonction pour convertir le mois en anglais
def french_to_english_month(month_french):
    months_mapping = {
        'janvier': 'January',
        'février': 'February',
        'mars': 'March',
        'avril': 'April',
        'mai': 'May',
        'juin': 'June',
        'juillet': 'July',
        'août': 'August',
        'septembre': 'September',
        'octobre': 'October',
        'novembre': 'November',
        'décembre': 'December'
    }
    return months_mapping.get(month_french.lower(), month_french)


# Fonction pour convertir la durée en minutes
def convert_duration(duration_str):
    if isinstance(duration_str, str):
        # Supprimer les espaces et diviser la chaîne en parties
        parts = duration_str.replace(' ', '').split('h')

        # Vérifier la présence des heures et des minutes
        if len(parts) == 2:
            hours = int(parts[0])
            minutes = 0 if 'min' not in parts[1] else int(parts[1].replace('min', ''))
            
            # Calculer la durée totale en minutes
            total_minutes = hours * 60 + minutes
            
            return int(total_minutes)
    
    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return float('nan')



# Fonction pouur extraire le nombre de critique de la presse 
def extract_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Utiliser isdigit() pour extraire uniquement les chiffres
        return np.nan if not critiques_str.split()[0].isdigit() else int(critiques_str.split()[0])
    else:
        return np.nan



# Fonction pour extraire le nombre de notes et le nombre de critiques
def extract_notes_and_critiques_count(critiques_str):
    if isinstance(critiques_str, str):
        # Trouver les nombres dans la chaîne
        numbers = [int(word) for word in critiques_str.split() if word.isdigit()]

        # Extraire le nombre de notes et de critiques en fonction de la longueur de la liste "numbers"
        if len(numbers) == 1:
            return numbers[0], np.nan
        elif len(numbers) == 2:
            return numbers[0], numbers[1]

    # Gérer le cas où la valeur est déjà un nombre ou ne peut pas être convertie
    return np.nan, np.nan

In [25]:
## Appliacation de nos fonctions


# Remplacer les chaînes "nan" par des valeurs NaN
df_film_ac['date'] = df_film_ac['date'].replace('nan', np.nan)

# Appliquer la fonction pour convertir le mois en anglais
df_film_ac['date'] = df_film_ac['date'].apply(lambda x: ' '.join([french_to_english_month(word) for word in str(x).split()]) if pd.notna(x) else np.nan)

# Utiliser dateutil.parser.parse pour convertir les dates en objets datetime
df_film_ac['date'] = df_film_ac['date'].apply(lambda x: parser.parse(x, dayfirst=True) if isinstance(x, str) else x)


# Appliquer la fonction de conversion pour la durée
df_film_ac['duree_minutes'] = df_film_ac['duree'].apply(convert_duration)

# Appliquer la fonction extract_notes_and_critiques_count
df_film_ac[['Nombre_de_notes_spectateurs', 'Nombre_de_critiques_spectateurs']] = df_film_ac['Critiques spectateurs'].apply(extract_notes_and_critiques_count).apply(pd.Series)

# Appliquer la fonction extract_critiques_count
df_film_ac['Nombre_de_critiques_presse'] = df_film_ac['Critiques presse'].apply(extract_critiques_count)

In [26]:
df_film_ac

,Titre original,Note press,Notes spectateur,Critiques presse,Critiques spectateurs,id allocine,Box office,Budget,date,duree,duree_minutes,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,Nombre_de_critiques_presse
0,Shutter Island,3.8,4.4,31 critiques,80471 notes dont 4605 critiques,132039,127770000.0,80 000 000 $,2010-02-24,2h 17min,137.0,80471.0,4605.0,31.0
1,Inception,4.1,4.5,24 critiques,110095 notes dont 7212 critiques,143692,290948208.0,160 000 000 $,2010-07-21,2h 28min,148.0,110095.0,7212.0,24.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,20 critiques,52676 notes dont 2887 critiques,126693,291377000.0,150 000 000 $,2010-11-24,2h 26min,146.0,52676.0,2887.0,20.0
3,Prince of Persia: The Sands of Time,2.6,3.1,22 critiques,26730 notes dont 2133 critiques,126678,89981000.0,200 000 000 $,2010-05-26,2h 06min,126.0,26730.0,2133.0,22.0
4,The Book of Eli,2.4,3.3,20 critiques,10503 notes dont 1144 critiques,128955,92524000.0,80 000 000 $,2010-01-20,1h 49min,109.0,10503.0,1144.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11816,Amarillo,NaN,NaN,NaN,NaN,294872,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11817,Chastise,NaN,NaN,NaN,NaN,295695,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11818,Tapawingo,NaN,NaN,NaN,NaN,296176,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN
11819,Mr. Birthday,NaN,NaN,NaN,NaN,298844,NaN,-,NaT,NaN,NaN,NaN,NaN,NaN


In [27]:
# Retrait des colonnes inutiles 
colonnes_a_retirer = ["Critiques presse","Critiques spectateurs","duree","Budget"]
df_film_ac = df_film_ac.drop(colonnes_a_retirer,axis=1)

In [29]:
# Retrait de tous les films sans critique spectateurs
# On garde que les films avec au moins plus de 5 commentaires

df_film_ac_clean = df_film_ac.dropna(subset=['Nombre_de_critiques_spectateurs'])
df_film_ac_clean = df_film_ac_clean[df_film_ac_clean['Nombre_de_critiques_spectateurs'] >= 5]

In [30]:
# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_film_ac_clean['mois_sortie'] = df_film_ac_clean['date'].dt.month
df_film_ac_clean['annee_sortie'] = df_film_ac_clean['date'].dt.year

In [31]:
df_film_ac_clean

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,Nombre_de_critiques_presse,mois_sortie,annee_sortie
0,Shutter Island,3.8,4.4,132039,127770000.0,2010-02-24,137.0,80471.0,4605.0,31.0,2.0,2010.0
1,Inception,4.1,4.5,143692,290948208.0,2010-07-21,148.0,110095.0,7212.0,24.0,7.0,2010.0
2,Harry Potter and the Deathly Hallows - Part 1,3.2,4.0,126693,291377000.0,2010-11-24,146.0,52676.0,2887.0,20.0,11.0,2010.0
3,Prince of Persia: The Sands of Time,2.6,3.1,126678,89981000.0,2010-05-26,126.0,26730.0,2133.0,22.0,5.0,2010.0
4,The Book of Eli,2.4,3.3,128955,92524000.0,2010-01-20,109.0,10503.0,1144.0,20.0,1.0,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...
11101,The Gateway,NaN,2.3,273584,NaN,2022-03-16,91.0,40.0,6.0,NaN,3.0,2022.0
11102,"Batman: The Long Halloween, Part Two",NaN,3.7,292641,NaN,2021-08-04,84.0,114.0,7.0,NaN,8.0,2021.0
11104,Mortal Kombat Legends: Battle of the Realms,NaN,3.3,294101,NaN,2021-09-15,80.0,50.0,6.0,NaN,9.0,2021.0
11117,Black As Night,NaN,1.6,287632,NaN,2021-10-01,87.0,53.0,6.0,NaN,10.0,2021.0


In [32]:
df_film_ac_clean.to_csv('df_film_ac_clean.csv', index=False)

### Scrapping des données sur TMDB

Le scrapping des données sur TMDB nécessite l'utilisation de l'API du site. Pour se faire, il a fallu créer une clé :

In [ ]:
key_api='d1d1413d8379729633d60e9f5cc4a730'

In [ ]:
## Fonctions

# Fonction pour récupérer l'id du film
def id_recup(titre):
    url_api=f"https://api.themoviedb.org/3/search/movie?api_key={key_api}&query={titre}" 
    req = requests.get(url_api)
    carte = req.json()

    ind=[]
    for film in range(len(carte['results'])):
        ind.append(carte['results'][film]['id'])
    return(ind)


# Fonction pour récuperer les infos du film à partir de l'id 
def df_avec_id(id):
    id_film= id
    url_new_api = f"https://api.themoviedb.org/3/movie/{id_film}?api_key={key_api}&language=en-US"
    req_new = requests.get(url_new_api)
    wb_new = req_new.json()
    
    
    #ajustement des données 
    if 'belongs_to_collection' in wb_new and wb_new['belongs_to_collection'] is not None:
        wb_new['belongs_to_collection'] = wb_new['belongs_to_collection']['name']

        
    # Ajustement des données pour la clé 'genres'
    if 'genres' in wb_new:
        wb_new['genres'] = ', '.join([x['name'] for x in wb_new['genres']])
    else:
        wb_new['genres'] = None

# Ajustement des données pour la clé 'production_companies'
    if 'production_companies' in wb_new:
        wb_new['production_companies'] = ', '.join([x['name'] for x in wb_new['production_companies']])
    else:
        wb_new['production_companies'] = None

# Ajustement des données pour la clé 'production_countries'
    if 'production_countries' in wb_new:
        wb_new['production_countries'] = ', '.join([x['name'] for x in wb_new['production_countries']])
    else:
        wb_new['production_countries'] = None

# Ajustement des données pour la clé 'spoken_languages'
    if 'spoken_languages' in wb_new:
        wb_new['spoken_languages'] = ', '.join([x['name'] for x in wb_new['spoken_languages']])
    else:
        wb_new['spoken_languages'] = None

    
    df=pd.DataFrame(wb_new, index=[0])
    
    return (df)


def get_movie_info(movie_id_list):
# Initialiser un DataFrame vide pour stocker les informations sur les films
    movie_df = pd.DataFrame()

    for movie_id in movie_id_list:
        # Utiliser la deuxième fonction pour obtenir les informations détaillées du film
        movie_info = df_avec_id(movie_id)

        # Vérifier si des informations ont été trouvées
        if movie_info is not None:
            # Ajouter les informations du film au DataFrame
            movie_df = pd.concat([movie_df, movie_info], ignore_index=True)

    return movie_df


def create_movie_list(movie_list):
    all_movies_df = pd.DataFrame()

    for movie_name in movie_list:
        movie_id_list = id_recup(movie_name)
        if movie_id_list:
            movie_info_df = get_movie_info(movie_id_list)
            # Ajouter les informations du film au DataFrame global
            all_movies_df = pd.concat([all_movies_df, movie_info_df], ignore_index=True)

    return all_movies_df



# Fonction qui prend en entrée une liste et qui renvoie 4 sous-liste de taille identique (à une division entière près) 
def diviser_liste(liste):
    taille = len(liste)
    quart = taille // 4
    partie1 = liste[:quart]
    partie2 = liste[quart:2*quart]
    partie3 = liste[2*quart:3*quart]
    partie4 = liste[3*quart:]
    return partie1, partie2, partie3, partie4

On récupère ensuite les films de la base d'Allociné nettoyé précédemment.

In [33]:
df_film_ac_clean= pd.read_csv("df_film_ac_clean.csv")
liste_films_ac = df_film_ac_clean['Titre original'].tolist()

len(liste_films_ac)

4194

On va séparer notre dataframe en 4 en vue de la récolte des données à l'aide de l'API de TMDB. En effet, on rencontrait une erreur liée aux nombres de requêtes lorsqu'on utilisait le data frame en entier (ou même si on le séparait en 2)

In [34]:
liste_films_ac_1, liste_films_ac_2, liste_films_ac_3, liste_films_ac_4 = diviser_liste(liste_films_ac)


# Affichage des quatres listes résultantes
print("Partie 1:", len(liste_films_ac_1))
print("Partie 2:", len(liste_films_ac_2))
print("Partie 3:", len(liste_films_ac_3))
print("Partie 4:", len(liste_films_ac_4))

Partie 1: 1048
Partie 2: 1048
Partie 3: 1048
Partie 4: 1050


In [ ]:
%%time 
df_film_ac_clean_1 = create_movie_list(liste_films_ac_1)


# CPU times: user 3min 37s, sys: 17.1 s, total: 3min 54s
# Wall time: 21min 6s

In [ ]:
%%time
df_film_ac_clean_2 = create_movie_list(liste_films_ac_2)

# CPU times: user 3min 54s, sys: 20.2 s, total: 4min 14s
# Wall time: 21min 41s

In [ ]:
%%time
df_film_ac_clean_3 = create_movie_list(liste_films_ac_3)

# CPU times: user 3min 52s, sys: 19.9 s, total: 4min 12s
# Wall time: 20min 36s

In [ ]:
%%time
df_film_ac_clean_4 = create_movie_list(liste_films_ac_4)

# CPU times: user 3min 43s, sys: 19.5 s, total: 4min 04s
# Wall time: 20min 56s

In [ ]:
dataframes = [df_film_ac_clean_1, df_film_ac_clean_2, df_film_ac_clean_3, df_film_ac_clean_4]

# Rassembler les DataFrames en un seul DataFrame
df_tmdb = pd.concat(dataframes, ignore_index=True)

# Sauvegarder le DataFrame tmdb en CSV
df_tmdb.to_csv('df_tmdb.csv', index=False)

# Afficher le DataFrame final
df_tmdb.head()

## Nettoyage de la bas TMDB

In [36]:
df_tmdb= pd.read_csv("df_tmdb.csv", engine="python")

In [38]:
noms_variables = df_tmdb.columns.tolist()
#print(noms_variables)

for i in noms_variables:
    print(i,":",df_tmdb[i].isnull().sum())

#on observe que les trois dernières colonnes peuvent être supprimés de la dataframe 
#trop de valeurs manquantes

colonnes_a_supprimer = ["success", "status_code", "status_message"]
df_tmdb_clean = df_tmdb.drop(columns=colonnes_a_supprimer)

adult : 6
backdrop_path : 15053
belongs_to_collection : 30493
budget : 181
genres : 5489
homepage : 26420
id : 161
imdb_id : 7521
original_language : 181
original_title : 161
overview : 1692
popularity : 187
poster_path : 4893
production_companies : 11739
production_countries : 8237
release_date : 2024
revenue : 213
runtime : 213
spoken_languages : 7419
status : 213
tagline : 22747
title : 213
video : 213
vote_average : 213
vote_count : 213
success : 34373
status_code : 34373
status_message : 34373


In [39]:
# Changement format release date 

df_tmdb_clean['release_date'] = pd.to_datetime(df_tmdb['release_date'], errors="coerce")
df_tmdb_clean["release_date"].value_counts()

release_date
2009-01-01    75
2011-01-01    71
2008-01-01    70
2014-01-01    68
2007-01-01    68
              ..
2021-08-21     1
1998-06-09     1
1950-05-19     1
1980-03-28     1
1983-09-16     1
Name: count, Length: 12586, dtype: int64

In [40]:
# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_tmdb_clean['mois_sortie'] = df_tmdb_clean['release_date'].dt.month
df_tmdb_clean['annee_sortie'] = df_tmdb_clean['release_date'].dt.year

In [41]:
df_tmdb_clean

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,mois_sortie,annee_sortie
0,False,/ecvy2kMxsJ60ej52beZ0F8EOGkL.jpg,NaN,80000000.0,"Drama, Thriller, Mystery",http://www.shutterisland.com/,11324.0,tt1130884,en,Shutter Island,...,138.0,"English, Deutsch",Released,Some places never let you go.,Shutter Island,False,8.201,22634.0,2.0,2010.0
1,False,NaN,NaN,0.0,Documentary,NaN,1157141.0,tt5445282,en,Shutter Island: Into the Lighthouse,...,21.0,English,Released,NaN,Shutter Island: Into the Lighthouse,False,0.000,0.0,6.0,2010.0
2,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,NaN,160000000.0,"Action, Science Fiction, Adventure",https://www.warnerbros.com/movies/inception,27205.0,tt1375666,en,Inception,...,148.0,"English, Français, 日本語, Kiswahili",Released,Your mind is the scene of the crime.,Inception,False,8.366,34886.0,7.0,2010.0
3,False,/JeGkRdNsOuMrgwBdtB0hp763MU.jpg,El Crack Collection,0.0,"Drama, Thriller",https://filmaxinternationalsales.com/film/the-...,613092.0,tt6793710,es,El crack cero,...,117.0,Español,Released,The prequel to the cult film El Crack,The Crack: Inception,False,6.700,36.0,10.0,2019.0
4,False,NaN,NaN,0.0,"Animation, Action, Thriller, Science Fiction",NaN,64956.0,tt5295894,en,Inception: The Cobol Job,...,14.0,English,Released,NaN,Inception: The Cobol Job,False,7.300,290.0,12.0,2010.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34374,False,NaN,NaN,0.0,"Mystery, Drama, Action",https://www.youtube.com/watch?v=dkgKvEnzUvk,738314.0,NaN,ar,كابوس والغيوم السوداء,...,9.0,العربية,Released,NaN,Nightmare and the Black Clouds,False,0.000,0.0,4.0,2020.0
34375,False,/kz4vvPMVWOP5d7133piLAslNh3o.jpg,NaN,0.0,"Drama, TV Movie",NaN,147227.0,tt0078211,de,Schwarz und weiß wie Tage und Nächte,...,103.0,Deutsch,Released,NaN,Black and White Like Day and Night,False,5.900,7.0,12.0,1978.0
34376,False,/zvirPUc2icShoJ2FeWKMi3bsZEe.jpg,NaN,0.0,"Comedy, Romance",NaN,389862.0,tt0098574,ru,В городе Сочи тёмные ночи,...,115.0,Pусский,Released,NaN,How Dark the Nights Are on the Black Sea,False,3.000,1.0,3.0,1989.0
34377,False,/fpprduGgOxKid7XtuQMyeYTl4sR.jpg,NaN,100000.0,Action,http://terpconnect.umd.edu/~dwilt/oronegro.htm,386475.0,tt0208336,es,La noche de San Juan: Santo en Oro negro,...,80.0,Español,Released,NaN,Night of San Juan: Santo in Black Gold,False,4.300,7.0,2.0,1977.0


### Merge de la base Allociné et de la base TMDB

In [35]:
# Séparation du mois et de l'année de sortie dans les deux bases

# Extraire le jour, le mois et l'année dans des colonnes distinctes
df_tmdb['mois_sortie'] = df_tmdb['release_date'].dt.month
df_tmdb['annee_sortie'] = df_tmdb['release_date'].dt.year

df_tmdb.head()

NameError: name 'df_tmdb' is not defined

### Nettoyage de la base finale

Tout comme pour la base Allociné, nous allons procéder au nettoyage de la base obtenue après le merge des deux bases.

In [44]:
df_film_ac_clean['titre_normalise'] = df_film_ac_clean['Titre original'].str.replace(r'\W', '', regex=True).str.lower()
df_tmdb_clean['titre_normalise'] = df_tmdb_clean['original_title'].str.replace(r'\W', '', regex=True).str.lower()

In [46]:
liste_titre_ac = df_film_ac_clean["titre_normalise"].tolist()
liste_titre_tmdb = df_tmdb_clean["titre_normalise"].tolist()

In [48]:
#nous commencons par prendre les elements qui se ressemblent dans les deux listes
liste_ressemblance = [element for element in liste_titre_tmdb if element in liste_titre_ac]
#len(nouvelle_liste)

On remarque à ce niveau que certains films apparaissent plusieurs fois dans nouvelle_liste du fait que de nombreux films sont sortis sous le meme nom 

In [51]:
# On commence par garder dans la base df_tmdb_clean que les films qui sont dans nouvelle_liste
df_filtre_tmdb = df_tmdb_clean[df_tmdb_clean['titre_normalise'].isin(liste_ressemblance)]


# On trie ensuite la base par ordre alphabetique des films et decroissants des dates
df_filtre_tmdb_trie = df_filtre_tmdb.sort_values(by=['titre_normalise', 'release_date'], ascending=[True, False])
df_filtre_tmdb_trie = df_filtre_tmdb_trie.reset_index(drop=True)

# Conservation des films sorties entre 2010 et 2021 (et valeurs manquantes aussi)
df_filtre_tmdb_trie = df_filtre_tmdb_trie[(df_filtre_tmdb_trie['annee_sortie'] >= 2010) & (df_filtre_tmdb_trie['annee_sortie'] <= 2021) | df_filtre_tmdb_trie['annee_sortie'].isna()]
df_filtre_tmdb_trie = df_filtre_tmdb_trie.reset_index(drop=True)

# On retire tous les doublons parfaits de la base 
df_filtre_tmdb_trie = df_filtre_tmdb_trie.drop_duplicates(keep='first')
df_filtre_tmdb_trie = df_filtre_tmdb_trie.reset_index(drop=True)

df_filtre_tmdb_trie.to_csv("df_tmdb_tri.csv", index=False)

Pour merge les deux bases, nous allons utiliser le nom du réalisateur comme clé primaire.

In [53]:
# On recupere les id des films sur la base tmdb pour recuperer le nom des realisateur de chaque film
liste_id = df_filtre_tmdb_trie["id"].tolist()
liste_id_entiers = [int(nombre) for nombre in liste_id]
len(liste_id_entiers)

7312

In [ ]:
%%time

def get_directors_for_movies(movie_ids, api_key):
    directors_dict = {}

    for movie_id in movie_ids:
        # Envoyer une requête GET à l'API TMDb pour obtenir les crédits du film
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={api_key}')

        # Vérifier si la requête a réussi (code 200)
        if response.status_code == 200:
            # Extraire le contenu JSON de la réponse
            json_data = response.json()

            # Filtrer la liste des membres de l'équipe pour ne conserver que les réalisateurs
            directors = [member.get('name') for member in json_data.get('crew', []) if member.get('job') == 'Director']

            # Ajouter l'association identifiant-réalisateur au dictionnaire
            directors_dict[movie_id] = directors if directors else None
        else:
            # En cas d'échec de la requête, ajouter une valeur manquante au dictionnaire
            directors_dict[movie_id] = None

    return directors_dict



In [ ]:
# Exemple d'utilisation avec une liste d'identifiants de films
api_key = "d1d1413d8379729633d60e9f5cc4a730"

# Appeler la fonction pour obtenir le dictionnaire des réalisateurs
directors_result = get_directors_for_movies(liste_id_entiers, api_key)

In [ ]:
# Ajouter une colonne 'nom_du_realisateur' en utilisant le dictionnaire
df_filtre_tmdb_trie['nom_du_realisateur'] = df_filtre_tmdb_trie['id'].map(directors_result)

# Afficher le DataFrame résultant
df_filtre_tmdb_trie.head()
df_filtre_tmdb_trie.to_csv("df_filtre_trie.csv", index=False)

On récupère ensuite le nom des réalisateurs sur Allociné

In [54]:
%%time

def get_director_name(url):
    # Envoyer une requête GET à l'URL de la page Allociné
    response = requests.get(url)

    # Vérifier si la requête a réussi (code 200)
    if response.status_code == 200:
        # Extraire le contenu HTML de la réponse
        html_content = response.content

        # Utiliser BeautifulSoup pour parcourir le HTML
        soup = BeautifulSoup(html_content, 'html.parser')

        # Trouver la balise contenant le nom du réalisateur
        director_tag = soup.find('div', class_='meta-body-direction')

        # Extraire le texte entre les balises
        director_name = director_tag.text.strip() if director_tag else None

        return director_name

    else:
        # En cas d'échec de la requête, retourner une valeur manquante
        return None

def get_director_names_from_urls(identifiants):
    director_names = {}

    for identifiant in identifiants:
        # Construire l'URL à partir de l'identifiant
        url = f'https://www.allocine.fr/film/fichefilm_gen_cfilm={identifiant}.html'

        # Appeler la fonction pour obtenir le nom du réalisateur
        nom_realisateur = get_director_name(url)

        # Ajouter le nom du réalisateur au dictionnaire
        director_names[identifiant] = nom_realisateur

    return director_names


CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 11.7 µs


In [ ]:
liste_identifiants = df_film_ac_clean["id allocine"].tolist()

resultat = get_director_names_from_urls(liste_identifiants)

# Afficher le résultat
#for identifiant, nom_realisateur in resultat.items():
    #print(f"Film {identifiant}: Réalisateur {nom_realisateur}")

print(len(resultat))
print(resultat)

In [ ]:
# Ajouter une colonne 'nom_du_realisateur' en utilisant le dictionnaire
df_film_ac_clean['nom_du_realisateur'] = df_film_ac_clean['id allocine'].map(resultat)

# Afficher le DataFrame résultant
df_film_ac_clean.head()
df_film_ac_clean.to_csv("df_film_ac_clean.csv", index=False)

In [ ]:
df_film_ac_clean['nom_du_realisateur'] = df_film_ac_clean['nom_du_realisateur'].str.replace('De\n', '')

In [ ]:
#ne pas mettre dans la meme cellule qu'avant

df_film_ac_clean['nom_du_realisateur'] = df_film_ac_clean['nom_du_realisateur'].str.replace('\n', '')
#df_film_avec_critiques.head()

In [ ]:
df_filtre_tmdb_trie['nom_du_realisateur'] = df_filtre_tmdb_trie['nom_du_realisateur'].apply(lambda x: x[0] if x else None)
#df_filtre_trie.head()

In [55]:
def remove_accents(input_str):
    accents = {
        'a': '[aáàâäãå]',
        'e': '[eéèêë]',
        'i': '[iíìîï]',
        'o': '[oóòôöõ]',
        'u': '[uúùûü]',
        'c': '[cç]',
        'n': '[nñ]'
    }
    for char, pattern in accents.items():
        input_str = re.sub(pattern, char, input_str)
    return input_str

In [ ]:
# Appliquer la fonction remove_accents à la colonne nom_du_realisateur
df_filtre_tmdb_trie['nom_du_realisateur_bis'] = df_filtre_tmdb_trie['nom_du_realisateur'].apply(lambda x: remove_accents(x) if pd.notnull(x) else x)
df_film_ac_clean['nom_du_realisateur_bis'] = df_film_ac_clean['nom_du_realisateur'].apply(lambda x: remove_accents(x) if pd.notnull(x) else x)

In [ ]:
#normalisation 

df_film_ac_clean['nom_du_realisateur_bis'] = df_film_ac_clean['nom_du_realisateur_bis'].str.replace(r'\W', '', regex=True).str.lower()
#df_film_avec_critiques.head()
df_filtre_tmdb_trie['nom_du_realisateur_bis'] = df_filtre_tmdb_trie['nom_du_realisateur_bis'].str.replace(r'\W', '', regex=True).str.lower()
#df_filtre_trie.head()


In [ ]:
df_film_ac_clean = df_film_ac_clean.sort_values(by=['titre_normalise', 'date'], ascending=[True, False])
df_film_ac_clean = df_film_ac_clean.reset_index(drop=True)
#df_film_avec_critiques.head(12)

In [ ]:
#conserve uniquement les correspondances
df_merged = pd.merge(df_film_ac_clean, df_filtre_tmdb_trie, left_on=['titre_normalise', 'nom_du_realisateur_bis'], right_on=['titre_normalise', 'nom_du_realisateur_bis'])
print(len(df_merged))

df_merged.to_csv("df_merged.csv", index=False)

#conserve les correspondances et tout du cote df_film_avec_critiques
#df_merged_bis= pd.merge(df_film_avec_critiques, df_filtre_trie, on=['titre_normalise', 'nom_du_realisateur_bis'], how='left')
#df_merged_bis.to_csv("df_merged_bis.csv", index=False)


### Scrapping des critiques sur Allocine

Maintenant que nous avons notre base finale, avec toutes les données, on peut faire le scrapping des critiques spectateurs sur Allociné grâce à l'id du film.

In [74]:
df_final_ac = pd.read_csv('df_merged.csv')
df_final_ac

,Titre original,Note press,Notes spectateur,id allocine,Box office,date,duree_minutes,Nombre_de_critiques_presse,Nombre_de_notes_spectateurs,Nombre_de_critiques_spectateurs,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,mois_sortie_y,annee_sortie_y,nom_du_realisateur_y
0,10 000 days,NaN,1.5,239331,NaN,NaN,91.0,NaN,32.0,7.0,...,English,Released,NaN,"10,000 Days",False,2.800,15.0,11.0,2014.0,Eric Small
1,100 Degrees Below Zero,NaN,1.2,221267,NaN,2015-04-21,89.0,NaN,110.0,51.0,...,English,Released,Cold as Hell,100 Degrees Below Zero,False,4.400,112.0,3.0,2013.0,R.D. Braunstein
2,10 Cloverfield Lane,3.6,3.6,231144,67880730.0,2016-03-16,103.0,20.0,9399.0,735.0,...,English,Released,Monsters come in many forms.,10 Cloverfield Lane,False,6.984,7456.0,3.0,2016.0,Dan Trachtenberg
3,10 Minutes Gone,NaN,1.5,266802,NaN,2020-01-10,96.0,NaN,170.0,23.0,...,English,Released,Keep your enemies close,10 Minutes Gone,False,5.358,204.0,9.0,2019.0,Brian A. Miller
4,10x10,NaN,2.0,251534,NaN,2018-11-29,87.0,NaN,447.0,54.0,...,English,Released,There are some secrets we cannot escape,10x10,False,5.200,508.0,4.0,2018.0,Suzi Ewing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3562,Zombeavers,NaN,1.9,226998,NaN,2015-02-17,85.0,NaN,444.0,66.0,...,English,Released,You'll be dammed!,Zombeavers,False,4.903,650.0,10.0,2014.0,Jordan Rubin
3563,Zombie Apocalypse,NaN,1.3,197632,NaN,2015-04-07,87.0,NaN,153.0,46.0,...,"English, Português, Pусский",Released,Mankind Survived... But Not Alive.,Zombie Apocalypse,False,5.300,217.0,10.0,2011.0,Nick Lyon
3564,Zombieland: Double Tap,3.1,3.5,176293,66643981.0,2019-10-30,99.0,23.0,5501.0,329.0,...,English,Released,NaN,Zombieland: Double Tap,False,6.951,5274.0,10.0,2019.0,Ruben Fleischer
3565,Zookeeper,1.0,2.2,86789,75800690.0,2011-08-17,99.0,2.0,735.0,102.0,...,English,Released,Welcome to his jungle.,Zookeeper,False,5.429,1539.0,7.0,2011.0,Frank Coraci


In [75]:
df_final_ac['Nombre_de_critiques_spectateurs'] = df_final_ac['Nombre_de_critiques_spectateurs'].astype(int)
#df_final_ac = df_final_ac.sort_values(by='Nombre_de_critiques_spectateurs', ascending=False)

somme_critique = df_final_ac['Nombre_de_critiques_spectateurs'].sum()

liste_id = df_final_ac['id allocine'].tolist()
nb_critique = df_final_ac['Nombre_de_critiques_spectateurs'].tolist()
print(somme_critique)

800488


In [76]:
%%time

def scrapping_critique(liste_id, nb_critique):
    assert len(liste_id) == len(nb_critique)
    data = []
    nb_page=1
    # Boucle sur les pages
    k=0
    for j in range(len(liste_id)):
        i=0
        page=0

        while i!=nb_critique[j]:
            base_url = f"https://www.allocine.fr/film/fichefilm-{liste_id[j]}/critiques/spectateurs/?page=".format()
            page+=1
            url_page_critique = f"{base_url}{page}"
            response_page_critique = requests.get(url_page_critique)
            
            if response_page_critique.status_code == 200:
                bs_page_critique = bs(response_page_critique.text, "html.parser")
                films_page_critique = bs_page_critique.findAll("div", attrs={'class': "hred review-card cf"})

                for critique in films_page_critique:
                    try:

                        note_critique = critique.find('span', class_='stareval-note').text.strip()

                        date_publication = critique.find('span', class_='review-card-meta-date light').text.strip()
                        critique = critique.find('div', class_='content-txt review-card-content').text.strip()

                        data.append([liste_id[j], note_critique, date_publication, critique])
                        i+=1
                        k+=1
                        print(f' {k} / {sum(nb_critique)} '.format())
                        df_critique = pd.DataFrame(data, columns=["id_allocine", "Note de la critique", "Date de publication", 'Critique'])
                    except Exception as e:
                        print(f"Une erreur s'est produite pour le commentaire {j} : {e}")
    
    return df_critique

CPU times: user 6 µs, sys: 1e+03 ns, total: 7 µs
Wall time: 11.4 µs


In [77]:
## On split nos deux listes en 8

def diviser_liste_en_8(liste):
    taille = len(liste)
    huitieme = taille // 8
    partie1 = liste[:huitieme]
    partie2 = liste[huitieme:2*huitieme]
    partie3 = liste[2*huitieme:3*huitieme]
    partie4 = liste[3*huitieme:4*huitieme]
    partie5 = liste[4*huitieme:5*huitieme]
    partie6 = liste[5*huitieme:6*huitieme]
    partie7 = liste[6*huitieme:7*huitieme]
    partie8 = liste[7*huitieme:]

    return partie1, partie2, partie3, partie4, partie5, partie6, partie7, partie8

In [78]:
%%time

liste_id_1, liste_id_2, liste_id_3, liste_id_4, liste_id_5, liste_id_6, liste_id_7, liste_id_8 = diviser_liste_en_8(liste_id)
nb_critique_1, nb_critique_2, nb_critique_3, nb_critique_4, nb_critique_5, nb_critique_6, nb_critique_7, nb_critique_8 = diviser_liste_en_8(nb_critique)
print(len(liste_id_1))
print(len(liste_id))

445
3567
CPU times: user 212 µs, sys: 24 µs, total: 236 µs
Wall time: 245 µs


In [80]:
%%time

scrapping_critique_1 = scrapping_critique(liste_id_1, nb_critique_1)
scrapping_critique_1.to_csv('df_scrapping_critique_1.csv', index=False)


 1 / 69306 
 2 / 69306 
 3 / 69306 
 4 / 69306 
 5 / 69306 
 6 / 69306 
 7 / 69306 
 8 / 69306 
 9 / 69306 
 10 / 69306 
 11 / 69306 
 12 / 69306 
 13 / 69306 
 14 / 69306 
 15 / 69306 
 16 / 69306 
 17 / 69306 
 18 / 69306 
 19 / 69306 
 20 / 69306 
 21 / 69306 
 22 / 69306 
 23 / 69306 
 24 / 69306 
 25 / 69306 
 26 / 69306 
 27 / 69306 
 28 / 69306 
 29 / 69306 
 30 / 69306 
 31 / 69306 
 32 / 69306 
 33 / 69306 
 34 / 69306 
 35 / 69306 
 36 / 69306 
 37 / 69306 
 38 / 69306 
 39 / 69306 
 40 / 69306 
 41 / 69306 
 42 / 69306 
 43 / 69306 
 44 / 69306 
 45 / 69306 
 46 / 69306 
 47 / 69306 
 48 / 69306 
 49 / 69306 
 50 / 69306 
 51 / 69306 
 52 / 69306 
 53 / 69306 
 54 / 69306 
 55 / 69306 
 56 / 69306 
 57 / 69306 
 58 / 69306 
 59 / 69306 
 60 / 69306 
 61 / 69306 
 62 / 69306 
 63 / 69306 
 64 / 69306 
 65 / 69306 
 66 / 69306 
 67 / 69306 
 68 / 69306 
 69 / 69306 
 70 / 69306 
 71 / 69306 
 72 / 69306 
 73 / 69306 
 74 / 69306 
 75 / 69306 
 76 / 69306 
 77 / 69306 
 78 / 69

KeyboardInterrupt: 

In [ ]:
%%time

scrapping_critique_2 = scrapping_critique(liste_id_2, nb_critique_2)
scrapping_critique_2.to_csv('df_scrapping_critique_2.csv', index=False)


In [ ]:
%%time

scrapping_critique_3 = scrapping_critique(liste_id_3, nb_critique_3)
scrapping_critique_3.to_csv('df_scrapping_critique_3.csv', index=False)


In [ ]:
%%time

scrapping_critique_4 = scrapping_critique(liste_id_4, nb_critique_4)
scrapping_critique_4.to_csv('df_crapping_critique_4.csv', index=False)


In [ ]:
%%time

scrapping_critique_5 = scrapping_critique(liste_id_5, nb_critique_5)
scrapping_critique_5.to_csv('df_scrapping_critique_5.csv', index=False)


In [ ]:
%%time

scrapping_critique_6 = scrapping_critique(liste_id_6, nb_critique_6)
scrapping_critique_6.to_csv('df_scrapping_critique_6.csv', index=False)


In [ ]:
%%time

scrapping_critique_7 = scrapping_critique(liste_id_7, nb_critique_7)
scrapping_critique_7.to_csv('df_scrapping_critique_7.csv', index=False)


In [ ]:
%%time

scrapping_critique_8 = scrapping_critique(liste_id_8, nb_critique_8)
scrapping_critique_8.to_csv('df_scrapping_critique_8.csv', index=False)


In [ ]:
## On merge les 4 dataframes en un
dfs = [scrapping_critique_1, scrapping_critique_2, scrapping_critique_3, scrapping_critique_4, scrapping_critique_5, scrapping_critique_6, scrapping_critique_7, scrapping_critique_8]

# Rassembler les DataFrames en un seul DataFrame
df_critiques = pd.concat(dfs, ignore_index=True)

# Sauvegarder le DataFrame des critiques en CSV
df_critiques.to_csv('df_critiques.csv', index=False)

# Afficher le DataFrame final
#df_critiques.head()